##### 1. INSTALL AND IMPORT DEPENDENCIES

In [46]:
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil,os
from pathlib import Path
import uuid
import json
import matplotlib.pyplot as plt
import time
import cv2
import labelme
from sklearn.model_selection import train_test_split
import wget

##### 2. COLLECT AND LABEL IMAGES

In [2]:
labels = ['thumbsup','thumbsdown','livelong','thankyou']
numberOfImages = 5

In [3]:
IMAGES_PATH = os.path.join('Tensorflow','workspace','images','collectedImages')

In [4]:
for label in labels:
    label_path = os.path.join(IMAGES_PATH,label)
    if not os.path.exists(label_path):
        os.makedirs(label_path,exist_ok=True)

In [6]:
cap = cv2.VideoCapture(0)
for label in labels:
    print(f'Collecting images for {label}')
    for i in range(numberOfImages):
        ret,frame = cap.read()
        image_path = os.path.join(IMAGES_PATH,label,f"{label}_{str(uuid.uuid1())}.jpg")
        cv2.imwrite(image_path,frame)
        cv2.imshow('frame',frame)
        time.sleep(1)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [10]:
labelImg_path = os.path.join('Tensorflow','labelImg')

if not os.path.exists(labelImg_path):
    os.makedirs(labelImg_path,exist_ok=True)
    !git clone https://github.com/HumanSignal/labelImg {labelImg_path}

Cloning into 'Tensorflow\labelImg'...


In [14]:
if os.name == 'nt':
    !cd {labelImg_path} && pyrcc5 -o libs/resources.py resources.qrc

In [17]:
! cd {labelImg_path} && python labelImg.py

Image:C:\Users\bkj\Documents\GitHub\AI PROJECTS\computer vision\sign language detection with tfod\Tensorflow\workspace\images\collectedImages\livelong\1f567895-40f7-11ee-b19d-ec5c68664d70.jpg -> Annotation:C:/Users/bkj/Documents/GitHub/AI PROJECTS/computer vision/sign language detection with tfod/Tensorflow/workspace/images/collectedImages/livelong\1f567895-40f7-11ee-b19d-ec5c68664d70.xml
Image:C:\Users\bkj\Documents\GitHub\AI PROJECTS\computer vision\sign language detection with tfod\Tensorflow\workspace\images\collectedImages\livelong\21c08c9f-40f7-11ee-a2a7-ec5c68664d70.jpg -> Annotation:C:/Users/bkj/Documents/GitHub/AI PROJECTS/computer vision/sign language detection with tfod/Tensorflow/workspace/images/collectedImages/livelong\21c08c9f-40f7-11ee-a2a7-ec5c68664d70.xml
Cancel creation.
Image:C:\Users\bkj\Documents\GitHub\AI PROJECTS\computer vision\sign language detection with tfod\Tensorflow\workspace\images\collectedImages\livelong\22f59f2a-40f7-11ee-a63f-ec5c68664d70.jpg -> Anno


Image:C:\Users\bkj\Documents\GitHub\AI PROJECTS\computer vision\sign language detection with tfod\Tensorflow\workspace\images\collectedImages\livelong\242b11a0-40f7-11ee-9a32-ec5c68664d70.jpg -> Annotation:C:/Users/bkj/Documents/GitHub/AI PROJECTS/computer vision/sign language detection with tfod/Tensorflow/workspace/images/collectedImages/livelong\242b11a0-40f7-11ee-9a32-ec5c68664d70.xml
Image:C:\Users\bkj\Documents\GitHub\AI PROJECTS\computer vision\sign language detection with tfod\Tensorflow\workspace\images\collectedImages\thankyou\14c5b76d-40f7-11ee-8a37-ec5c68664d70.jpg -> Annotation:C:/Users/bkj/Documents/GitHub/AI PROJECTS/computer vision/sign language detection with tfod/Tensorflow/workspace/images/collectedImages/thankyou\14c5b76d-40f7-11ee-8a37-ec5c68664d70.xml
Image:C:\Users\bkj\Documents\GitHub\AI PROJECTS\computer vision\sign language detection with tfod\Tensorflow\workspace\images\collectedImages\thankyou\15f9560d-40f7-11ee-91ba-ec5c68664d70.jpg -> Annotation:C:/Users/

##### 3. SPLIT TO TRAIN TEST

In [37]:
for folder in ['train','test']:
    folder_path = os.path.join(os.path.join('Tensorflow','workspace','images'),folder)
    os.makedirs(folder_path,exist_ok=True)

for label in labels:
    label_folder = os.path.join(IMAGES_PATH,label)
    filenames = [files for files in Path(label_folder).glob('*.jpg')]
    train,test = train_test_split(filenames,test_size=0.2,random_state=42)

    for files in train:
        label_abspath = files.with_suffix('.xml')
        shutil.copy(files,os.path.join(os.path.join('Tensorflow','workspace','images'),'train'))
        shutil.copy(label_abspath,os.path.join(os.path.join('Tensorflow','workspace','images'),'train'))

    for files in test:
        label_abspath = files.with_suffix('.xml')
        shutil.copy(files,os.path.join(os.path.join('Tensorflow','workspace','images'),'test'))
        shutil.copy(label_abspath,os.path.join(os.path.join('Tensorflow','workspace','images'),'test'))

##### 4. DOWNLOAD TF MODELS PRETRAINED MODELS FROM TENSORFLOW MODEL ZOO

In [38]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [43]:
paths ={
    'WORKSPACE_PATH': os.path.join('Tensorflow','workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow','workspace','annotation'),
    'IMAGE_PATH': os.path.join('Tensorflow','workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow','workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow','workspace','pre-trained_models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'export'),
    'TFJS_PATH': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'tfjsexport'),
    'TFLITE_PATH': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'tfliteexport'),
    'PROTOC_PATH': os.path.join('Tensorflow','protoc')
}

In [44]:
files = {
    'PIPELINE_CONFIG': os.path.join('Tensorflow','workspace','models',CUSTOM_MODEL_NAME,'pipeline.config'),
    'TF_RECORD_SCRIPTS': os.path.join(paths['SCRIPTS_PATH'],TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'],LABEL_MAP_NAME)
}

In [45]:
for path in paths:
    os.makedirs(paths[path],exist_ok=True)

In [47]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'],'research','object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODELS_PATH']}

##### 5. GET PROTOC FILE

In [52]:
protoc_url = 'https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip'
wget.download(protoc_url)
!move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
!cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'],'bin'))
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
!cd Tensorflow/models/research/slim && pip install -e .

        1 file(s) moved.
        1 file(s) copied.

zip_safe flag not set; analyzing archive contents...


object_detection.core.__pycache__.densepose_ops.cpython-310: module references __file__

running build



object_detection.core.__pycache__.preprocessor.cpython-310: module MAY be using inspect.stack

...

copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\center_net_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\eval_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\flexible_grid_anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\fpn_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\graph_rewriter_pb2.py -> build\lib\object_detection\protos
copying ob

  Preparing metadata (setup.py): started

  Preparing metadata (setup.py): finished with status 'done'

  Running setup.py develop for slim



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [53]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'],'research','object_detection','builders','model_builder_tf2_test.py')
!python {VERIFICATION_SCRIPT}

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Running tests under Python 3.10.2: c:\Users\bkj\AppData\Local\Programs\Python\Python310\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0908 00:35:54.445844  6468 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.


W0908 00:35:54.459847  6468 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W0908 00:35:54.463843  6468 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W0908 00:35:54.472842  6468 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W0908 00:35:54.486842  6468 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layer

##### 6. GET PRETRAINED MODEL

In [58]:
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME + '.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


##### 7. CREATE LABEL MAP

In [73]:
labels = [{'name':'thumbsup','id':1},{'name':'thumbsdown','id':2},{'name':'thankyou','id':3},{'name':'livelong','id':4}]

with open(files['LABELMAP'],'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')


##### 8 CREATE TF RECORDS

In [68]:
if not os.path.exists(files['TF_RECORD_SCRIPTS']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}
    

Cloning into 'Tensorflow\scripts'...


In [74]:
!python {files['TF_RECORD_SCRIPTS']} -x {os.path.join(paths['IMAGE_PATH'],'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'],'train.record')}
!python {files['TF_RECORD_SCRIPTS']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 


Successfully created the TFRecord file: Tensorflow\workspace\annotation\train.record
Successfully created the TFRecord file: Tensorflow\workspace\annotation\test.record


##### 9. COPY MODEL CONFIG TO TRAINING FOLDER

In [80]:
!copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}


        1 file(s) copied.


##### 10 UPDATE CONFIG FOR TRANSFER LEARNING

In [81]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [82]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [83]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [84]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]


In [85]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)  

##### 11. TRAIN THE MODEL

In [86]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

!{command}

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
W0908 01:31:38.270160 25552 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0908 01:31:38.303111 25552 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I0908 01:31:38.310177 2

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0908 01:31:47.982257 25552 deprecation.py:364] From c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0908 01:31:52.086045 25552 deprecation.py:364] From c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1176: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf

##### 12 EVALUATE THE MODEL

In [87]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

!{command}

creating index...

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 


index created!

TensorFlow Addons (TFA) has ended development and introduction of new features.


creating index...

...

I0908 02:36:18.856985 16276 dataset_builder.py:162] Reading unweighted datasets: ['Tensorflow\\workspace\\annotation\\test.record']


INFO:tensorflow:Reading record datasets for input file: ['Tensorflow\\workspace\\annotation\\test.record']

 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.667



I0908 02:36:18.858978 16276 dataset_builder.py:79] Reading record datasets for input file: ['Tensorflow\\workspace\\annotation\\test.record']
INFO:tensorflow:Number of filenames to read: 1
I0908 02:36:18.858978 16276 dataset_builder.py:80] Number of filenames to read: 1
W0908 02:36:18.858978 16276 dataset_builder.py:86] num_readers has been reduced to 1 to match input file shards.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.deterministic`.
W0908 02:36:18.869004 16276 deprecation.py:364] From c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\object_detection\builders\dataset_builder.py:100: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_le

Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0908 02:36:25.522186 16276 deprecation.py:364] From c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
Use `tf.cast` instead.
W0908 02:36:27.171948 16276 deprecation.py:364] From c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.

W0908 02:36:34.239149 16276 module_wrapper.py:149] From c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\object_detection\builders\optimizer_builder.py:124: The na

##### 13. LOAD TRAIN MODEL FROM CHECKPOINT

In [89]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [94]:
# load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'],is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'],'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image,shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image,shapes)
    detections = detection_model.postprocess(prediction_dict,shapes)
    return detections

##### 14. DETECT FROM AN IMAGE

In [108]:
import matplotlib
matplotlib.use('Qt5Agg') 

In [96]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])


In [98]:
IMAGEFILE_PATH = os.path.join(paths['IMAGE_PATH'],'test','03cb5480-40f7-11ee-b964-ec5c68664d70.jpg')
IMAGEFILE_PATH

'Tensorflow\\workspace\\images\\test\\03cb5480-40f7-11ee-b964-ec5c68664d70.jpg'

In [109]:
img = cv2.imread(IMAGEFILE_PATH)
image_np  = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np,0),dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.8,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.savefig('output_image.png') 
plt.show()

In [114]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while cap.isOpened(): 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break